In [1]:
# import json data to pandas
import pandas as pd
import json
import os
import numpy as np
import asyncio
import aiohttp

# rerank

In [7]:
valid_rank_data = pd.read_json('data_and_code/ytx_data/validation_prediction_alldochybrid_rerank.jsonl', lines=True)
valid_rank_data

,question,answer,document_id
0,when did the british first land in north america,NaN,"[570, 9798, 8471, 1592, 3101]"
1,when did the 1st world war officially end,NaN,"[10722, 1820, 4428, 2227, 10565]"
2,who's the girl that plays the new wonder woman,NaN,"[7556, 10786, 5970, 10380, 8669]"
3,who is the director of the cia today,NaN,"[11721, 7747, 7816, 393, 589]"
4,who plays ben in the new fantastic four,NaN,"[9192, 8635, 5557, 2700, 11223]"
...,...,...,...
995,who did morgan freeman play in driving miss daisy,NaN,"[7099, 2987, 7797, 8561, 8372]"
996,who played mr kidd in diamonds are forever,NaN,"[7690, 1297, 1587, 11109, 4066]"
997,when does nathan find out about chuck and blair,NaN,"[2452, 299, 3759, 3717, 6231]"
998,when did the first push to start car come out,NaN,"[8015, 5964, 5035, 5139, 1565]"


In [8]:
document_data = pd.read_json('data_and_code/ytx_data/processed_markdown.jsonl', lines=True)
document_plain = pd.read_json('data_and_code/ytx_data/processed_plain.jsonl', lines=True)
document_plain

,doc_id,text
0,0,Email marketing - Wikipedia Email marketing Em...
1,1,The Mother ( How I Met Your Mother ) - wikiped...
2,2,Human fertilization - wikipedia Human fertiliz...
3,3,List of National Football League career quarte...
4,4,Roanoke Colony - wikipedia Roanoke Colony `` L...
...,...,...
12133,12133,Jeepers Creepers ( 2001 film ) - wikipedia Jee...
12134,12134,Death Wish ( 2018 film ) - wikipedia Death Wis...
12135,12135,X-Men : Apocalypse - wikipedia X-Men : Apocaly...
12136,12136,Gulliver 's Travels - wikipedia Gulliver 's Tr...


In [9]:
# get max number of tokens in one document
max(document_data['text'].apply(lambda x: len(x.split(' ')))), \
    max(document_plain['text'].apply(lambda x: len(x.split(' '))))

(25123, 38117)

In [10]:
def idx2content(idx, document_data=document_data):
    """
    get document content by index
    """
    return document_data['text'][idx]

In [18]:
# rerank
from QA_System.backbone.rerank import ReRanker


def rerank_func(valid_rank_data, reranker: ReRanker):
    input_top_n = 20
    docid_origin_list = valid_rank_data['document_id'].apply(lambda x: x[:input_top_n])
    query_list = valid_rank_data['question']
    doc_content_origin_list = docid_origin_list.apply(lambda x: [idx2content(int(i)) for i in x])
    # transfer to list
    doc_content_origin_list = doc_content_origin_list.tolist()
    query_list = query_list.tolist()
    docid_origin_list = docid_origin_list.tolist()
    # combine query and document, format: [query, list of documents content]
    query_document_list = []
    for i in range(len(query_list)):
        query_document_list.append(
            [
                query_list[i],
                doc_content_origin_list[i]
            ]
)
    # rerank
    responses = reranker.async_send_requests_simple(
        query_document_list, use_progress_bar=False, concurrency=5, top_n = 5, 
        model = 'Pro/BAAI/bge-reranker-v2-m3',
    )
    _, indices = reranker.extract_json(responses)
    # indices is the relative index of the document in the original list
    # transform to the original document id
    # docid_origin_list = list[list[int]]
    # indices = list[list[int]]
    for i in range(len(indices)):
        for j in range(len(indices[i])):
            indices[i][j] = docid_origin_list[i][indices[i][j]]
    return indices

In [ ]:
import nest_asyncio
import time
nest_asyncio.apply()

api_key = "sk-"
reranker = ReRanker(api_token=api_key)
all_indices = []
# process 100 samples each time
batch_size = 100
for i in range(0, len(valid_rank_data), batch_size):
    print(f"Processing {i} to {i + batch_size}")
    indices = rerank_func(valid_rank_data[i:i + batch_size], reranker)
    all_indices.extend(indices)
    time.sleep(60)
# indices = rerank_func(valid_rank_data, reranker)

Processing 0 to 100
Processing 100 to 200
Processing 200 to 300
Processing 300 to 400
Processing 400 to 500
Processing 500 to 600
Processing 600 to 700
Processing 700 to 800
Processing 800 to 900
Processing 900 to 1000


In [31]:
# replace the attribute of 'document_id' in valid_rank_data
valid_rank_data['document_id'] = all_indices

In [30]:
all_indices_np.shape

(1000, 5)

In [33]:
# save valid_rank_data
valid_rank_data.to_json('data_and_code/ytx_data/validation_prediction_alldochybrid_rerank.jsonl', orient='records', lines=True)

# llm

In [2]:
from QA_System.backbone.llm import QwenChatClient
import nest_asyncio


In [3]:
valid_rank_data = pd.read_json('data_and_code/ytx_data/validation_prediction_alldochybrid_rerank.jsonl', lines=True)
valid_rank_data


,question,answer,document_id
0,when did the british first land in north america,NaN,"[570, 9798, 8471, 1592, 3101]"
1,when did the 1st world war officially end,NaN,"[10722, 1820, 4428, 2227, 10565]"
2,who's the girl that plays the new wonder woman,NaN,"[7556, 10786, 5970, 10380, 8669]"
3,who is the director of the cia today,NaN,"[11721, 7747, 7816, 393, 589]"
4,who plays ben in the new fantastic four,NaN,"[9192, 8635, 5557, 2700, 11223]"
...,...,...,...
995,who did morgan freeman play in driving miss daisy,NaN,"[7099, 2987, 7797, 8561, 8372]"
996,who played mr kidd in diamonds are forever,NaN,"[7690, 1297, 1587, 11109, 4066]"
997,when does nathan find out about chuck and blair,NaN,"[2452, 299, 3759, 3717, 6231]"
998,when did the first push to start car come out,NaN,"[8015, 5964, 5035, 5139, 1565]"


In [11]:
def get_answer(client: QwenChatClient, valid_rank_data=valid_rank_data):
    """
    get answer from QwenChatClient
    """
    # get the first 5 document id
    docid_list = valid_rank_data['document_id']
    # get the question
    query_list = valid_rank_data['question']
    doc_content_list = docid_list.apply(lambda x: idx2content(int(x[0])))
    # transfer to list
    doc_content_list = doc_content_list.tolist()
    query_list = query_list.tolist()
    # combine query and document, format: [query, documents content]
    query_context_list = []
    for i in range(len(query_list)):
        query_context_list.append(
            (
                query_list[i],
                doc_content_list[i]
            )
        )
    # get the answer
    results = client.batch_request_async_simple(
        query_context_list=query_context_list, 
        concurrency=5, 
        # model="Qwen/Qwen2.5-7B-Instruct", 
        model='Pro/Qwen/Qwen2.5-7B-Instruct',
        n = 1)
    extracted_answers = client.extract_answer(results)
    # extracted_answers format: list[list[str]]
    extracted_answers = [x[0] for x in extracted_answers]
    return extracted_answers

In [ ]:
# generate answers
import nest_asyncio
import time
nest_asyncio.apply()

api_token = "sk-"
client = QwenChatClient(api_token=api_token)
# answers = get_answer(client, valid_rank_data=valid_rank_data[2:10])

all_answers = []
# process 100 samples each time
# 如果超出max_tokens，返回"None"
batch_size = 100
for i in range(0, len(valid_rank_data), batch_size):
    print(f"Processing {i} to {i + batch_size}")
    indices = get_answer(client, valid_rank_data[i:i + batch_size])
    all_answers.extend(indices)
    time.sleep(30)


Processing 0 to 100
Extracting answers...


100%|██████████| 100/100 [00:00<00:00, 72944.42it/s]


Processing 100 to 200
Failed to request QwenChat API: {"code":20015,"message":"max_total_tokens (32831) must be less than or equal to max_seq_len (32768)","data":null}
we will return answer as "None"
Extracting answers...


100%|██████████| 100/100 [00:00<00:00, 214104.34it/s]


Processing 200 to 300
Extracting answers...


...

100%|██████████| 100/100 [00:00<00:00, 147220.22it/s]


Processing 800 to 900
Extracting answers...


100%|██████████| 100/100 [00:00<00:00, 170154.32it/s]


Processing 900 to 1000
Failed to request QwenChat API: {"code":50508,"message":"System is too busy now. Please try again later.","data":null}
we will return answer as "None"
Failed to request QwenChat API: {"code":50508,"message":"System is too busy now. Please try again later.","data":null}
we will return answer as "None"
Failed to request QwenChat API: {"code":50508,"message":"System is too busy now. Please try again later.","data":null}
we will return answer as "None"
Failed to request QwenChat API: {"code":50508,"message":"System is too busy now. Please try again later.","data":null}
we will return answer as "None"
Failed to request QwenChat API: {"code":50508,"message":"System is too busy now. Please try again later.","data":null}
we will return answer as "None"
Failed to request QwenChat API: {"code":50508,"message":"System is too busy now. Please try again later.","data":null}
we will return answer as "None"
Failed to request QwenChat API: {"code":50508,"message":"System is too 

100%|██████████| 100/100 [00:00<00:00, 228572.43it/s]


In [13]:
all_answers_np = np.array(all_answers)
all_answers_np.shape

(1000,)

In [14]:
# replace the attribute of 'answer' in valid_rank_data
valid_rank_data['answer'] = all_answers

In [16]:
# save valid_rank_data
valid_rank_data.to_json('data_and_code/ytx_data/validation_prediction_alldochybrid_rerank.jsonl', orient='records', lines=True)

In [ ]:
# client = QwenChatClient(api_token=api_token)
# query_context_list = [("What is the best Chinese large model?", 
#                         "The Chinese large model industry is growing rapidly and has attracted a lot of attention. The industry is expected to face both opportunities and challenges in the coming years. The best model is Qwen."),
#                         ("What is the best Chinese large model?",
#                         "The Chinese large model industry is growing rapidly and has attracted a lot of attention. The industry is expected to face both opportunities and challenges in the coming years. The best model is deepseek."),
#                         ("What is the best Chinese large model?",
#                             "The Chinese large model industry is growing rapidly and has attracted a lot of attention. The industry is expected to face both opportunities and challenges in the coming years. The best model is hunyuan."),
#                         ]
# results = client.batch_request_async_simple(
#     query_context_list=query_context_list, 
#     concurrency=5, model="Qwen/Qwen2.5-7B-Instruct", 
#     n = 3)
# extracted_answers = client.extract_answer(results)